In [1]:
from edge_sensor.sdr import airt
import labbench as lb

lb.show_messages('debug')
sdr = airt.AirTSource()

In [2]:
sdr.open()

 DEBUG  2024-06-28 16:45:39,730.731 • AirTSource(): connecting
 DEBUG  2024-06-28 16:45:45,675.676 • AirTSource(): connected
 DEBUG  2024-06-28 16:45:53,081.082 • AirTSource(): opened


In [3]:
sdr.autosample(3710e6, 15.36e6, 10e6, shift='left')
sdr.analysis_filter

 DEBUG  2024-06-28 16:45:53,158.158 • AirTSource(): 20833333.333333332 (Hz) → sample_rate
 DEBUG  2024-06-28 16:45:53,163.164 • AirTSource(): 20833333.333333332 (Hz) → backend_sample_rate
 DEBUG  2024-06-28 16:45:53,166.166 • AirTSource(): 15360000.0 (Hz) → sample_rate
 DEBUG  2024-06-28 16:45:53,173.174 • AirTSource(): 10000000.0 (Hz) → analysis_bandwidth


{'window': 'hamming',
 'fft_size': 3125,
 'fft_size_out': 2304,
 'frequency_shift': 'left',
 'passband': (-10473333.333333332, -473333.333333333),
 'fs': 20833333.333333332}

In [14]:
sdr.channel_enabled(True, channel=0)
try:
    iq = sdr.acquire(0.09*15.36e6)
    %timeit -n1 -r1 sdr.acquire(0.09*15.36e6)
finally:
    sdr.channel_enabled(False, channel=0)

 DEBUG  2024-06-28 16:48:53,707.708 • AirTSource(): True  → channel_enabled


<class 'cupy.ndarray'>
<class 'cupy.ndarray'> <class 'cupy.ndarray'>
<class 'cupy.ndarray'>
<class 'cupy.ndarray'> <class 'cupy.ndarray'>
121 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


 DEBUG  2024-06-28 16:48:54,016.017 • AirTSource(): False  → channel_enabled


In [15]:
sdr.channel_enabled(True, channel=0)


 DEBUG  2024-06-28 16:49:11,498.499 • AirTSource(): True  → channel_enabled


In [ ]:
iq = sdr.acquire(0.09*15.36e6)


In [ ]:
# import sklearn.linear_model
from edge_sensor.sdr import airt
from channel_analysis import waveform
from iqwaveform import fourier

duration = 0.1
center_frequency_Hz = 763e6
source.analysis_bandwidth = 10e6
source.sample_rate = 15.36e6

fs_sdr, lo_offset, resample_kws = fourier.design_cola_frequency_shift(fs_base=125e6, fs_target=source.sample_rate, bw=source.analysis_bandwidth)

print('connecting...')
capture = airt.AirTCapture(frequency_Hz=center_frequency_Hz+2*lo_offset, source.sample_rate=fs_sdr, gain_dB=0)

try:
    print('acquiring...')
    raw_iq = capture.acquire(int(duration * fs_sdr))
    resample_kws['frequency_shift'] = 'right'
    iq = fourier.ola_filter(raw_iq, extend=True, **resample_kws)
finally:
    print('done')
    capture.close()

# # Option 2: No resample and shift
# print('connecting...')
# capture = airt.AirTCapture(frequency_Hz=center_frequency_Hz, source.sample_rate=source.sample_rate, gain_dB=0)

# try:
#     print('acquiring...')
#     iq = capture.acquire(int(duration * source.sample_rate))
# finally:
#     print('done')
#     capture.close()

In [19]:
# Monday dan: this fails on rounding problems because it's looking at the actual sample rate, not the request.
 
from channel_analysis import waveform

analysis_spec = {
    'power_time_series': {
        'detector_period': 10e-3,
        'detectors': ('rms', 'peak')
    },
    'cyclic_channel_power': {
        'cyclic_period': 10e-3,
        'detector_period': 1e-3 / 15 / 4,
        'detectors': ('rms', 'peak'),
        'cyclic_statistics': ('min', 'mean', 'max'),
    },
    'persistence_spectrum': {
        'window': 'flattop',
        'resolution': 15e3,
        'quantiles': [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 1],
        'truncate': False
    },
    'amplitude_probability_distribution': {
        'power_low': -105,
        'power_high': -60,
        'power_count': 221,  # 0.25 dB resolution
    },
    # 'iq_waveform': {
    #     'start_time_sec': 0,
    #     'stop_time_sec': 100e-3
    # }
}

filter_spec = {}

dataset = waveform.from_spec(
    iq,
    sdr,
    analysis_spec=analysis_spec,
)

 DEBUG  2024-06-28 16:50:37,256.257 • AirTSource(): sample_rate → 20833492.279052734  (Hz)
 DEBUG  2024-06-28 16:50:37,259.260 • AirTSource(): backend_sample_rate → 20833492.279052734  (Hz)
 DEBUG  2024-06-28 16:50:37,263.263 • AirTSource(): sample_rate → 15360117.1875  (Hz)


ValueError: bin period (0.01 s) must be multiple of waveform sample period (6.510366996508307e-08)

In [ ]:
import iqwaveform
from channel_analysis import figures
import matplotlib.pyplot as plt
from iqwaveform import powtodB

plt.style.use('iqwaveform.ieee_double_column')

# fig, ax = plt.subplots()
# pvt = dataset.power_time_series
# pvt.plot.line(x='time_elapsed', ax=ax)
# figures.label_axis(ax.xaxis, pvt, 'time_elapsed')
# figures.label_axis(ax.yaxis, pvt, tick_units=False)
# figures.label_legend(ax, pvt, 'power_detector')

# fig, ax = plt.subplots()
# apd = dataset.amplitude_probability_distribution
# apd.plot.line(x='channel_power', ax=ax)
# ax.set_yscale('gamma-qq', k=1, db_ordinal=True)
# figures.label_axis(ax.xaxis, apd, 'channel_power', tick_units=False)
# figures.label_axis(ax.yaxis, apd)

# fig, ax = plt.subplots()
# cyclic_power = dataset.cyclic_channel_power
# time = cyclic_power.cyclic_lag
# for i, detector in enumerate(cyclic_power.power_detector.values):
#     a = cyclic_power.sel(power_detector=detector)
#     ax.plot(time, powtodB(a.sel(cyclic_statistic='mean')), color=f'C{i}')

# for i, detector in enumerate(cyclic_power.power_detector.values):
#     a = cyclic_power.sel(power_detector=detector)
#     ax.fill_between(
#         time,
#         powtodB(a.sel(cyclic_statistic='min')),
#         powtodB(a.sel(cyclic_statistic='max')),
#         color=f'C{i}',
#         alpha=0.25,
#         lw=0,
#         rasterized=True,
#     )

# figures.label_axis(ax.xaxis, cyclic_power, 'cyclic_lag')
# figures.label_axis(ax.yaxis, cyclic_power, tick_units=False)
# figures.label_legend(ax, cyclic_power, 'power_detector')

fig, ax = plt.subplots()
spectrum = dataset.persistence_spectrum
spectrum.plot.line(x='baseband_frequency', ax=ax)
figures.label_axis(ax.xaxis, spectrum, 'baseband_frequency')
figures.label_axis(ax.yaxis, spectrum, tick_units=False)
figures.label_legend(ax, spectrum, 'persistence_statistic')

In [ ]:
spectrum.sel(persistence_statistic='0.5').idxmax()

In [ ]:
lo_offset*(2304/3125)